In [1]:
import tensorflow as tf

import numpy as np
import os
import time

2025-08-12 06:47:43.909988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754981264.178525      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754981264.254271      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
path_to_file = '../input/lyrics-generation/lyrics_dataset.txt'

In [3]:
# Opening the text file in read mode and standard encoding it
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 404640 characters


In [4]:
text[:100]

"I hate you for what you did\r\nAnd I miss you like a little kid\r\nI faked it every time\r\nBut that's alr"

In [5]:
vocab = sorted(set(text))
print(f'vocab size: {len(vocab)} few chars {vocab[:10]} type: {type(np.array(vocab))}')

vocab size: 98 few chars ['\n', '\r', ' ', '!', '"', '&', "'", '(', ')', '*'] type: <class 'numpy.ndarray'>


In [6]:
# token encoder
char2i = {c:i for i,c in enumerate(vocab)}
# token decoder
i2char = np.array(vocab)

encoded_text = np.array([char2i[c] for c in text])
print(text[100:200])
encoded_text[100:200]

ight
I can hardly feel anything
I hardly feel anything at all
You gave me fifteen hundred
To see


array([62, 60, 61, 73,  1,  0, 35,  2, 56, 54, 67,  2, 61, 54, 71, 57, 65,
       78,  2, 59, 58, 58, 65,  2, 54, 67, 78, 73, 61, 62, 67, 60,  1,  0,
       35,  2, 61, 54, 71, 57, 65, 78,  2, 59, 58, 58, 65,  2, 54, 67, 78,
       73, 61, 62, 67, 60,  2, 54, 73,  2, 54, 65, 65,  1,  0, 51, 68, 74,
        2, 60, 54, 75, 58,  2, 66, 58,  2, 59, 62, 59, 73, 58, 58, 67,  2,
       61, 74, 67, 57, 71, 58, 57,  1,  0, 46, 68,  2, 72, 58, 58])

In [7]:
print('{ ===========>')
for char,_ in zip(char2i, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2i[char]))
print('  ...\n==========>}')

{ ===========>
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '&' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  ...
==========>}


In [8]:
SEQ_LEN = 100
BATCH = 64
examples_per_epoch = len(text) // (SEQ_LEN + 1)
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
for i in char_dataset.take(5):
  print(i2char[i.numpy()] , end = "")


I hat

2025-08-12 06:48:05.620483: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text
    
sequences = char_dataset.batch(SEQ_LEN+1, drop_remainder=True)
dataset = sequences.map(split_input_target)

In [10]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(i2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(i2char[target_example.numpy()])))

Input data:  "I hate you for what you did\r\nAnd I miss you like a little kid\r\nI faked it every time\r\nBut that's alr"
Target data: " hate you for what you did\r\nAnd I miss you like a little kid\r\nI faked it every time\r\nBut that's alri"


In [11]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [12]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units 
rnn_units = 512

In [13]:
class GatedRecurrentUnit(tf.keras.layers.Layer):
    def __init__(self, units, return_sequences=False, **kwargs):
        super().__init__()
        self.units = units
        self.return_sequences = return_sequences
        self.name="CustomGatedRU"

    def build(self, input_shape):
        input_dim = input_shape[-1]
        initializer = tf.keras.initializers.GlorotUniform() #xavier initialization
        # relevance gate
        self.W_rc = self.add_weight(shape=(self.units, self.units), initializer=initializer, name='W_rc')
        self.W_rx = self.add_weight(shape=(input_dim, self.units), initializer=initializer, name='W_rx')
        self.b_r = self.add_weight(shape=(self.units,), initializer='zeros', name='b_r')

        # Update gate
        self.W_uc = self.add_weight(shape=(self.units, self.units), initializer=initializer, name='W_uc')
        self.W_ux = self.add_weight(shape=(input_dim, self.units), initializer=initializer, name='W_ux')
        self.b_u = self.add_weight(shape=(self.units,), initializer='zeros', name='b_u')

        # potential new candidate
        self.W_cc = self.add_weight(shape=(self.units, self.units), initializer=initializer, name='W_cc')
        self.W_cx = self.add_weight(shape=(input_dim, self.units), initializer=initializer, name='W_cx')
        self.b_c = self.add_weight(shape=(self.units,), initializer='zeros', name='b_c')

        super().build(input_shape)


    @tf.function
    def call(self, inputs, initial_state=None):
        #input : Batch, sequence/context len, Embedding
        batch_size = tf.shape(inputs)[0]
        seq_len = tf.shape(inputs)[1]
        if initial_state is None:
            h_t = tf.zeros(shape=(batch_size, self.units))
        else:
            h_t = initial_state

        outputs = tf.TensorArray(tf.float32, size=seq_len)

        for t in tf.range(seq_len):
            x_t = inputs[:, t, :]
            relevance_gate = tf.sigmoid((x_t @ self.W_rx) + (h_t @ self.W_rc) + self.b_r)
            update_gate = tf.sigmoid((x_t @ self.W_ux) + (h_t @ self.W_uc) + self.b_u)
            potential_new_candidate = tf.tanh((x_t @ self.W_cx) + relevance_gate*(h_t @ self.W_uc) + self.b_c)
            h_t = update_gate*potential_new_candidate + (1-update_gate)*h_t
            outputs = outputs.write(t, h_t)

        outputs = outputs.stack()
        # Transpose to the standard Keras output shape: (batch_size, sequence_length, units)
        outputs = tf.transpose(outputs, perm=[1, 0, 2])
        if self.return_sequences:
            return outputs
        else:
            # Return only the final hidden state
            return h_t

    def get_config(self):
        """Enables the layer to be saved and loaded."""
        config = super().get_config()
        config.update({
            "units": self.units,
            "return_sequences": self.return_sequences
        })
        return config

        
        

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
  
    GatedRecurrentUnit(rnn_units,
                        return_sequences=True, name='custom_gru'),

    tf.keras.layers.Dense(vocab_size,activation='relu'),
    
    tf.keras.layers.Dropout(0.2),
  ])
  return model

In [15]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 98) # (batch_size, sequence_length, vocab_size)


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, 100, 256)         │        25,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ CustomGatedRU                   │ (64, None, 512)        │     1,181,184 │
│ (GatedRecurrentUnit)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, None, 98)         │        50,274 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (64, None, 98)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,256,546 (4.79 MB)

 Trainable params: 1,256,546 (4.79 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print(sampled_indices.shape)
print("Input: \n", repr("".join(i2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(i2char[sampled_indices ])))


(100,)
Input: 
 "121)\r\nWe ain't hit a rave in a while (London City)\r\nSo take me back to London (Woo)\r\nBass high, midd"

Next Char Predictions: 
 '!\'Vg{HP—2—P‘Iky];w¨‘?¨-\u2005rgYYa hw\'nBv\u205fG;u2¡F4íUKu6zMD9\n93¡wg\u205fd;£;!2")5–…í-T0&zHRMáMh"B\'TFé2{ej“J\x93;KLQ'


In [19]:
i2char[[10, 20, 37]]

array([',', '6', 'K'], dtype='<U1')

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

model.compile(optimizer='adam', loss=loss)


Prediction shape:  (64, 100, 98)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5832276


In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [22]:
EPOCHS=25 
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/25


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['sequential/CustomGatedRU/W_cc'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


62/62 ━━━━━━━━━━━━━━━━━━━━ 61s 893ms/step - loss: 3.6660
Epoch 2/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 856ms/step - loss: 2.8243
Epoch 3/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 82s 857ms/step - loss: 2.6476
Epoch 4/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 53s 850ms/step - loss: 2.5151
Epoch 5/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 858ms/step - loss: 2.4170
Epoch 6/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 876ms/step - loss: 2.3392
Epoch 7/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 56s 901ms/step - loss: 2.2900
Epoch 8/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 854ms/step - loss: 2.2273
Epoch 9/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 861ms/step - loss: 2.1835
Epoch 10/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 857ms/step - loss: 2.1552
Epoch 11/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 875ms/step - loss: 2.1225
Epoch 12/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 873ms/step - loss: 2.0743
Epoch 13/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 876ms/step - loss: 2.0412
Epoch 14/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 871ms/step - loss: 2.0039
Epoch 15/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 870ms/step 

In [23]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.build(tf.TensorShape([1, None]))
model.load_weights("./training_checkpoints/ckpt_25.weights.h5")
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (1, None, 256)         │        25,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ CustomGatedRU                   │ (1, None, 512)         │     1,181,184 │
│ (GatedRecurrentUnit)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (1, None, 98)          │        50,274 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (1, None, 98)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,256,546 (4.79 MB)

 Trainable params: 1,256,546 (4.79 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
def generate_text(model, chars_to_generate , temp , start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = chars_to_generate

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2i[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temp

  # Here batch size == 1
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(i2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [25]:
from numpy import arange

chars_to_generate = 500 

print(generate_text(model , chars_to_generate , 0.7 , start_string=u"Flower "))

Flower bon tUn bum s who6 linthe th i–hee it diveryoshousa ked mit al we

Dy myx w bril p ouryowe'tI s fe2 l id youmeromy
Oh‘{ulie warin

Al trUrJan I thoBat've byo y tou"Nou Ke

An d whi y f y gh tstkA wal
Ap “I bethe2  an se we l wow find mare linout kecke t br, chare !
I to y y blid s shongil the st
Shin: d—I whe
AntouKe wnd n s{Nouthourged fid s9

Sabrave don's loveNoplith ththjupe ldond kihel a2

I ckeve
An ary we sin"
Anelily, ing me waled thedyou p s/
OranKeldOhinghe ay ousetFr
